In [1]:
from firedrake import *
from mesh import OriginBoxMesh

/home/firedrake/firedrake/lib/python3.12/site-packages/pytools/__init__.py:2447: UserWarning: unable to find git revision
  warn("unable to find git revision", stacklevel=1)


In [6]:
# read a msh file
# mesh = Mesh("cube.msh",dim=3)
PI = 3.14159265358979323846
L = PI
origin = -PI/2
N = 20
mesh = OriginBoxMesh(N, N, N, L, L, L, originX=origin, originY=origin, originZ=origin)

# plot mesh

# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import axes3d
# fig, axes = plt.subplots(1, 1, subplot_kw={'projection': '3d'})
# triplot(mesh, axes=axes)
# axes.legend();

# create function space

degV = 1
V = FunctionSpace(mesh, "CG", degV)

u = TrialFunction(V)
v = TestFunction(V)

# change the coordinates of the mesh
x, y, z = SpatialCoordinate(mesh)
mu1, mu2, mu3 = 0.05, 0.22, 0.1

ex = exp(-((x-mu1)**2.0 + (y-mu2)**2.0 + ((z-mu3)**2.0))/2)
fct_uex = ex * sin(2*x) * sin(2*y) * sin(2*z)
u_ex = Function(V).interpolate(fct_uex)

VTKFile("u_ex.pvd").write(u_ex)

ex = exp(-((x-mu1)**2.0 + (y-mu2)**2.0 + ((z-mu3)**2.0))/2)
fct = (4.0*(-mu1 + x)**1.0*sin(2*y)*sin(2*z)*cos(2*x) - 1.0*(-mu1 + x)**2.0*sin(2*x)*sin(2*y)*sin(2*z) + 4.0*(-mu2 + y)**1.0*sin(2*x)*sin(2*z)*cos(2*y) - 1.0*(-mu2 + y)**2.0*sin(2*x)*sin(2*y)*sin(2*z) + 4.0*(-mu3 + z)**1.0*sin(2*x)*sin(2*y)*cos(2*z) - 1.0*(-mu3 + z)**2.0*sin(2*x)*sin(2*y)*sin(2*z) + 15.0*sin(2*x)*sin(2*y)*sin(2*z))*ex

f = Function(V).interpolate(fct)

a = inner(grad(u), grad(v))*dx
L = f*v*dx

g = Constant(0.0)
bc0 = DirichletBC(V.sub(0), g, "on_boundary")
# bc1 = DirichletBC(W.sub(0), as_vector([0.0, sin(5*x)]), 4)

sol = Function(V)
solve(a == L, sol, bcs=bc0)

VTKFile("sol.pvd").write(sol)

In [7]:


# compute the relative L2 error

error = sqrt(assemble((sol - u_ex)**2*dx)) / sqrt(assemble(u_ex**2*dx))
print("Error = ", error)

Error =  0.052203483070750636
